In [1]:
import random

In [2]:
random.seed(42)

# SPDZ

In [3]:
#
# Use e.g. https://www.compilejava.net/
#
#import java.util.*;
#import java.math.*;
#
#public class Entrypoint
#{
#  public static void main(String[] args)
#  {
#    BigInteger q = BigInteger.probablePrime(128, new Random());    
#    BigInteger inverse = new BigInteger("10000000000").modInverse(q);
#    System.out.println(q);
#    System.out.println(inverse);
#  }
#}

BASE = 10

#PRECISION_INTEGRAL = 1
#PRECISION_FRACTIONAL = 6
#Q = 10000019

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert Q > BASE**PRECISION

## Sharing and reconstruction

In [4]:
def share(secret, field=Q):
    first  = random.randrange(field)
    second = (secret - first) % field
    return [first, second]

def reconstruct(shares, field=Q):
    return sum(shares) % field

## Linear operations

In [5]:
def add(x, y, field=Q):
    return [ (xi + yi) % field for xi, yi in zip(x, y) ]

def sub(x, y, field=Q):
    return [ (xi - yi) % field for xi, yi in zip(x, y) ]

def scalar_add(x, k, field=Q):
    return add(x, [k, 0], field)

def scalar_sub(x, k, field=Q):
    return sub(x, [k, 0], field)

def scalar_mul(x, k, field=Q):
    return [ (xi * k) % field for xi in x ]

a = share(5)
b = share(7)
c = add(a, b)
assert reconstruct(c) == 12

## Multiplication

In [6]:
def generate_multiplication_triple(field=Q):
    a = random.randrange(field)
    b = random.randrange(field)
    c = a * b % field
    return share(a, field), share(b, field), share(c, field)

(a, b, c) = generate_multiplication_triple()
assert reconstruct(a) * reconstruct(b) % Q == reconstruct(c)

In [7]:
def mul(x, y, triple, field=Q):
    a, b, c = triple
    # local masking
    v = sub(x, a, field)
    w = sub(y, b, field)
    # communication: the players simultanously send their shares to the other
    epsilon = reconstruct(v, field)
    delta = reconstruct(w, field)
    # local combination
    r = epsilon * delta % field
    s = scalar_mul(b, epsilon, field)
    t = scalar_mul(a, delta, field)
    return add(s, add(t, scalar_add(c, r, field), field), field)

x = share(2)
y = share(5)
z = mul(x, y, generate_multiplication_triple())
assert reconstruct(z) == 10

## Exponentiation

In [8]:
def generate_exponentiation_triple(exponent):
    a = random.randrange(Q)
    return [ share(pow(a, e, Q)) for e in range(1, exponent+1) ]

a, aa = generate_exponentiation_triple(2)
assert(reconstruct(a) * reconstruct(a) % Q == reconstruct(aa))

a, aa, aaa = generate_exponentiation_triple(3)
assert(reconstruct(a) * reconstruct(a) * reconstruct(a) % Q == reconstruct(aaa))

In [9]:
from scipy.misc import comb as binom
from functools import reduce

ONE = [1,0] # constant sharing of 1

def exp(x, exponent, triple):
    # local masking
    v = sub(x, triple[0])
    # communication: the players simultanously send their share to the other
    epsilon = reconstruct(v)
    # local combination
    cs = [ binom(exponent, k, exact=True) for k in range(exponent+1) ]
    es = [ pow(epsilon, e, Q) for e in reversed(range(exponent+1)) ]
    bs = [ONE] + triple
    return reduce(add, ( scalar_mul(b, c * e) for c,e,b in zip(cs, es, bs) ))

x = share(4)
y = exp(x, 2, generate_exponentiation_triple(2))
z = exp(x, 10, generate_exponentiation_triple(10))
assert reconstruct(y) == pow(4, 2, Q)
assert reconstruct(z) == pow(4, 10, Q)

## Fixed-point encoding

In [10]:
def encode(rational, precision_fractional=PRECISION_FRACTIONAL, field=Q):
    upscaled = int(rational * BASE**precision_fractional)
    field_element = upscaled % field
    return field_element

def decode(field_element, precision_fractional=PRECISION_FRACTIONAL, field=Q):
    upscaled = field_element if field_element <= field/2 else field_element - field
    rational = upscaled / BASE**precision_fractional
    return rational

In [11]:
# using trick from SecureML paper that only requires a local operation

def truncate(x, amount=PRECISION_FRACTIONAL, field=Q):
    y0 = x[0] // BASE**amount
    y1 = field - ((field - x[1]) // BASE**amount)
    return [y0, y1]

In [12]:
x = share(encode(.5))

a = truncate(mul(x, x, generate_multiplication_triple()))
b = truncate(mul(a, a, generate_multiplication_triple()))
v = truncate(mul(b, b, generate_multiplication_triple()))
assert decode(reconstruct(v)) == (.5)**8

d = mul(x, x, generate_multiplication_triple())
e = mul(d, d, generate_multiplication_triple())
w = mul(e, e, generate_multiplication_triple())
print(decode(reconstruct(w)))
#assert decode(reconstruct(w)) == (.5)**8

1.34658068183981e+30


# Up and down sharing

In [13]:
# TODO don't redefine Q here!

Q = 1122700157294029049
P = 293973345475167247070445277780365744413

In [14]:
# TODO verify boundaries (not least for mask)

PRECISION = 6
KAPPA = 4

def generate_statistical_mask():
    return random.randrange(Q) % BASE**(2*PRECISION+1 + KAPPA)

def generate_zero_triple(field):
    return share(0, field)

def upshare(x, large_zero_triple):
    # map to positive range
    x = scalar_add(x, BASE**(2*PRECISION+1), Q)
    # player 0
    r = generate_statistical_mask()
    e = (x[0] + r) % Q
    y0 = (large_zero_triple[0] - r) % P
    # player 1
    xr = (e + x[1]) % Q
    y1 = (large_zero_triple[1] + xr) % P
    # combine
    y = [y0, y1]
    y = scalar_sub(y, BASE**(2*PRECISION+1), P)
    return y

def downshare(x, small_zero_triple):
    # map to positive range
    x = scalar_add(x, BASE**(2*PRECISION+1), P)
    # player 0
    r = generate_statistical_mask()
    e = (x[0] + r) % P
    y0 = (small_zero_triple[0] - r) % Q
    # player 1
    xr = (e + x[1]) % P
    y1 = (small_zero_triple[1] + xr) % Q
    # combine
    y = [y0, y1]
    return scalar_sub(y, BASE**(2*PRECISION+1), Q)

x = share(encode(-.5, PRECISION, Q), Q)

y = mul(x, x, generate_multiplication_triple(Q), Q)
y = mul(y, y, generate_multiplication_triple(Q), Q)
y = truncate(y, 4*PRECISION-PRECISION, Q)
assert decode(reconstruct(y, Q), PRECISION, Q) != (-.5)**4

x = upshare(x, generate_zero_triple(P))
y = mul(x, x, generate_multiplication_triple(P), P)
y = mul(y, y, generate_multiplication_triple(P), P)
y = truncate(y, 4*PRECISION-PRECISION, P)
y = downshare(y, generate_zero_triple(Q))
assert decode(reconstruct(y, Q), PRECISION, Q) == (-.5)**4

## Wrap-up

In [15]:
def multiplication_triple_generator():
    while True:
        yield generate_multiplication_triple()

In [16]:
multiplication_triple_source = multiplication_triple_generator()

class SecureRational(object):
    
    def secure(secret):
        z = SecureRational()
        z.shares = share(encode(secret))
        return z
    
    def reveal(self):
        return decode(reconstruct(self.shares))
    
    def __repr__(self):
        return "SecureRational(%f)" % self.reveal()
    
    def __add__(x, y):
        z = SecureRational()
        z.shares = add(x.shares, y.shares)
        return z
    
    def __sub__(x, y):
        z = SecureRational()
        z.shares = sub(x.shares, y.shares)
        return z
    
    def __mul__(x, y):
        z = SecureRational()
        triple = multiplication_triple_source.__next__()
        z.shares = truncate(mul(x.shares, y.shares, triple))
        return z
    
    def __pow__(x, e):
        # TODO
        z = SecureRational.secure(1)
        for _ in range(e):
            z = z * x
        return z

In [17]:
x = SecureRational.secure(.5)
y = SecureRational.secure(-.25)

z = x * y
assert z.reveal() == (.5) * (-.25)

z = x**8
assert z.reveal() == (.5)**8

In [18]:
class OpenRational(object):
    
    def __init__(self, secret):
        self.secret = secret
    
    def secure(secret):
        return OpenRational(secret)
    
    def reveal(self):
        return self.secret
    
    def __repr__(self):
        return "OpenRational(%f)" % self.reveal()
    
    def __add__(x, y):
        return OpenRational(x.secret + y.secret)
    
    def __sub__(x, y):
        return OpenRational(x.secret - y.secret)
    
    def __mul__(x, y):
        return OpenRational(x.secret * y.secret)
    
    def __pow__(x, e):
        z = OpenRational(1)
        for _ in range(e):
            z = z * x
        return z

In [19]:
x = OpenRational.secure(.5)
y = OpenRational.secure(-.25)

z = x * y
assert z.reveal() == (.5) * (-.25)

z = x**8
assert z.reveal() == (.5)**8

# Dump

## Security of mul

In [20]:
a, b, c = generate_multiplication_triple(Q)

x = share(3, Q)
y = share(4, Q)

In [21]:
e = sub(x, a, Q)
d = sub(y, b, Q)

epsilon = reconstruct(e, Q)
delta = reconstruct(d, Q)

r = epsilon * delta % Q
s = scalar_mul(b, epsilon, Q)
t = scalar_mul(a, delta, Q)

z = add(s, add(t, scalar_add(c, r, Q), Q), Q)

In [22]:
z0_real = (c[0] + b[0] * epsilon + a[0] * delta + epsilon * delta) % Q
z0_sim = (12 - z[1]) % Q

assert z0_real == z0_sim

## Security of add

In [23]:
x = share(2, Q)
y = share(5, Q)
z = add(x, y, Q)

In [24]:
z0_real = (x[0] + y[0]) % Q
z0_sim = (7 - z[1]) % Q

assert z0_real == z0_sim